# Web Scraping Housing Information
_This file web scrapes housing registration information at Columbia University._

In [58]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_all_elements_located as presence, all_of
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

In [59]:
# login credentials
uni = '<UNI>'
pwd = '<Password>'

In [60]:
# links for buildings I'm interested in
bldLnks = [
    "https://orsview.cuf.columbia.edu/Details.aspx?b=Broadway&s=&c=&u=&l=4",
    "https://orsview.cuf.columbia.edu/Details.aspx?b=Carlton&s=&c=&u=&l=109",
    "https://orsview.cuf.columbia.edu/Details.aspx?b=Harmony&s=&c=&u=&l=108",
    "https://orsview.cuf.columbia.edu/Details.aspx?b=McBain&s=&c=&u=&l=32",
    "https://orsview.cuf.columbia.edu/Details.aspx?b=Schapiro&s=&c=&u=&l=35",
    "https://orsview.cuf.columbia.edu/Details.aspx?b=Watt&s=&c=&u=&l=6",
    "https://orsview.cuf.columbia.edu/Details.aspx?b=Wien&s=&c=&u=&l=7",
    "https://orsview.cuf.columbia.edu/Details.aspx?b=600%20West%20113&s=&c=&u=&l=3"
]

In [61]:
options = Options()
options.add_argument('--headless')

In [62]:
def scrape(bldLnks, uni, pwd):
    '''
    Scrapes 2024 housing registration data and returns as a pandas dataframe.
    Note: Accessing the housing page requires Columbia login with DUO push notification.
    This takes around 10s, and requires manual approval through DUO mobile app.

    Parameters
    ----------
    bldLnks : list, list of links to building pages
    uni : str, UNI for login
    pwd : str, password for login

    Returns
    -------
    pd.DataFrame, housing registration data
    '''
    url = "https://orsview.cuf.columbia.edu/Summary.aspx"
    driver = webdriver.Chrome(options)
    wait = WebDriverWait(driver, 20)
    driver.get(url)

    # proceed to login page
    wait.until(presence((By.TAG_NAME, "input")))
    loginBtn = driver.find_element(By.XPATH, '//input[@type="submit"]')
    time.sleep(0.5)
    loginBtn.click()

    # login
    wait.until(EC.element_to_be_clickable((By.XPATH, '//input[@id="username"]'))).send_keys(uni)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//input[@id="password"]'))).send_keys(pwd)
    submitBtn = driver.find_element(By.XPATH, '//input[@name="submit"]')
    time.sleep(1)
    driver.execute_script("arguments[0].click()", submitBtn)

    # send duomobile push
    time.sleep(1)
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@id='duo_iframe']")))
    wait.until(EC.element_to_be_clickable((By.XPATH, "//button[normalize-space()='Send Me a Push']"))).click()
    time.sleep(1)
    wait.until(EC.url_changes(driver.current_url))

    # go to each building page and add to dataframe
    housing = [0] * len(bldLnks)

    for i in range(len(bldLnks)):
        driver.get(bldLnks[i])
        time.sleep(0.5)

        page = driver.page_source
        soup = bs(page, "html.parser")

        table = soup.find('table', {'class': 'filter'})
        housing[i] = pd.read_html(str(table))[0]
    
    driver.close()

    return pd.concat(housing)

In [63]:
# retrieve housing info
housing = scrape(bldLnks, uni, pwd)

In [64]:
# some pre-processing
to_drop = [col for col in housing.columns if 'Unnamed' in col]
housing.drop(to_drop, axis='columns', inplace=True)

housing_no_rec = housing[~housing.Building.str.contains('records')]


# get all buildings, configurations, and room types
all_buildings = housing_no_rec['Building'].unique()
all_configurations = housing_no_rec['Configuration'].unique()
all_room_types = housing_no_rec['Room Type'].unique()

In [65]:
# get available housing
available = housing_no_rec.loc[housing_no_rec['Status'] != 'Selected']

# get available buildings, configurations, and room types
buildings = available['Building'].unique()
configurations = available['Configuration'].unique()
room_types = available['Room Type'].unique()

In [67]:
# useful filtering methods
def search(df, col1, ls1, col2, ls2):
    df1 = df[df[col1].isin(ls1)]
    df2 = df1[df1[col2].isin(ls2)]
    return df2[['Building', 'Configuration', 'Room Name', 'Room Type']]

def search_bld_room(df, ls1=buildings, ls2=room_types):
    return search(df, 'Building', ls1, 'Room Type', ls2)

In [72]:
# example search
blds = ['McBain']
rooms = ['Double W/T']
search_bld_room(available, blds)

,Building,Configuration,Room Name,Room Type
0,McBain,1-Corridor Single,MCB 204,Single
1,McBain,1-Corridor Single,MCB 206,Single
2,McBain,1-Corridor Single,MCB 209,Single
3,McBain,1-Corridor Single,MCB 210,Single
4,McBain,1-Corridor Single,MCB 216,Single
...,...,...,...,...
380,McBain,2-Corridor Double,MCB 828,Double
381,McBain,2-Corridor Double,MCB 829,Double
382,McBain,2-Corridor Double,MCB 829,Double
383,McBain,2-Corridor Double,MCB 831,Double
